In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from importlib import reload
import matplotlib.pyplot as plt

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import model_params as params
import utils.resnet_orig as resnet
import utils.gmm_helpers as gmm_helpers

from tensorboardX import SummaryWriter
%load_ext line_profiler

In [2]:
def show_stats(model, device, model_params):
    print(tt.test(model, device, model_params.test_loader, min_conf=.101)[0])
    df = ev.evaluate(model, device, model_params.data_name, model_params.loaders)
    return df

def evaluate(model_params, device, file_joint, file_base, file_gmm):
    model_joint = torch.load('SavedModels/' + file_joint).to(device)
    model_joint.eval()

    df_joint = show_stats(model_joint, device, model_params)
    df_joint_base = show_stats(model_joint.base_model, device, model_params)

    base_model = torch.load('SavedModels/base/' + file_base)
    gmm = torch.load('SavedModels/GMM/' + file_gmm)

    lam = model_joint.loglam
    model = models.RobustModel(base_model, gmm, lam, dim=model_params.dim).to(device)
    model.train()

    df_sep = show_stats(model, device, model_params)
    df_sep_base = show_stats(model.base_model, device, model_params)
 
    df1 = pd.concat([df_sep_base, df_sep], axis=1, keys=['Base', 'GMM'])
    df2 = pd.concat([df_joint_base, df_joint], axis=1, keys=['Base', 'GMM'])
    return df1, df2 

In [3]:
device = torch.device('cuda:3')
dataset = 'MNIST'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCAMNIST_lam0.0_n100_lr0.001_lrgmm0.001_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_MNIST_lr0.001_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_MNIST_n100_data_used60000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

0.9944
0.995


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.995
0.9951
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &  FPR@95 &       MMC &     AUROC &  FPR@95 \\
DataSet     &           &           &         &           &           &         \\
\midrule
orig        &  0.991786 &         - &       - &  0.991171 &         - &       - \\
FMNIST      &  0.690625 &  0.979117 &  0.1705 &  0.275835 &  0.991394 &  0.0718 \\
EMNIST      &  0.819703 &  0.897694 &  0.4125 &  0.805450 &  0.896055 &  0.4052 \\
GrayCIFAR10 &  0.481831 &  0.996559 &  0.0045 &  0.100124 &  0.999699 &       0 \\
Noise       &  0.124669 &  0.999997 &       0 &  0.104590 &  0.999548 &       0 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &  FPR@95 &       MMC &     AUROC &  FPR@95 \\
DataSet     &           &           &         &           &           &         \\
\midrule
orig        &  0.991447 &  

In [4]:
device = torch.device('cuda:3')
dataset = 'FMNIST'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCAFMNIST_lam0.0_n100_lr0.1_lrgmm1e-05_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_FMNIST_lr0.1_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_FMNIST_n100_data_used60000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `

0.9505
0.9513


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.9491
0.951
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &     FPR@95 &       MMC &     AUROC &  FPR@95 \\
DataSet     &           &           &            &           &           &         \\
\midrule
orig        &  0.983704 &         - &          - &  0.983028 &         - &       - \\
MNIST       &  0.678978 &  0.973733 &     0.1204 &  0.678978 &  0.972996 &  0.1204 \\
EMNIST      &  0.659204 &  0.975426 &     0.1025 &  0.659204 &  0.974687 &  0.1025 \\
GrayCIFAR10 &  0.864589 &  0.917606 &     0.4992 &  0.190792 &  0.991474 &  0.0448 \\
Noise       &  0.502705 &  0.987545 &  0.0435937 &  0.339738 &  0.994042 &  0.0125 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &     FPR@95 &       MMC &     AUROC &     FPR@95 \\
DataSet     &           &           &            &           &           &            \\
\mi

In [5]:
device = torch.device('cuda:3')
dataset = 'SVHN'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCASVHN_lam0.0_n100_lr0.1_lrgmm1e-06_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_SVHN_lr0.01_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_SVHN_n100_data_used50000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

Using downloaded and verified file: ../data/train_32x32.mat
Using downloaded and verified file: ../data/train_32x32.mat
Using downloaded and verified file: ../data/test_32x32.mat


/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `

0.9694222495390289
Using downloaded and verified file: ../data/test_32x32.mat
0.969460663798402
Using downloaded and verified file: ../data/test_32x32.mat


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.9663106945298094
Using downloaded and verified file: ../data/test_32x32.mat
0.9673862937922557
Using downloaded and verified file: ../data/test_32x32.mat
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &    FPR@95 &       MMC &     AUROC &     FPR@95 \\
DataSet   &           &           &           &           &           &            \\
\midrule
orig      &  0.984496 &         - &         - &  0.984469 &         - &          - \\
CIFAR10   &  0.713052 &  0.945034 &    0.2088 &  0.567529 &  0.957556 &     0.1624 \\
CIFAR100  &  0.712355 &  0.943244 &    0.2113 &  0.529989 &   0.95784 &     0.1544 \\
LSUN\_CR   &  0.715110 &  0.944422 &  0.226667 &  0.158671 &  0.995344 &  0.0166667 \\
Imagenet- &  0.715440 &  0.943598 &    0.2181 &  0.209036 &  0.988872 &     0.0421 \\
Noise     &  0.641172 &  0.965957 &  0.137542 &  0.637791 &   0.96632 &   0.136315 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule

In [6]:
device = torch.device('cuda:3')
dataset = 'CIFAR10'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCACIFAR10_lam0.0_n100_lr0.1_lrgmm1e-05_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_CIFAR10_lr0.1_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_CIFAR10_n100_data_used50000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

Using downloaded and verified file: ../data/test_32x32.mat


/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `

0.945
0.9456


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.9395
0.9438
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &     FPR@95 &       MMC &     AUROC &     FPR@95 \\
DataSet   &           &           &            &           &           &            \\
\midrule
orig      &  0.970253 &         - &          - &  0.969627 &         - &          - \\
SVHN      &  0.661050 &  0.952938 &     0.0993 &  0.661057 &  0.952185 &     0.0992 \\
CIFAR100  &  0.792091 &  0.873726 &     0.3476 &  0.789369 &  0.873341 &     0.3463 \\
LSUN\_CR   &  0.738767 &  0.917583 &       0.24 &  0.727435 &  0.919578 &   0.233333 \\
Imagenet- &  0.787178 &  0.868608 &     0.3414 &  0.724067 &  0.878997 &     0.3126 \\
Noise     &  0.633381 &  0.960255 &  0.0782031 &  0.631463 &  0.960091 &  0.0711719 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &    FPR@95 &       MMC &     AUROC &    FPR@95

In [7]:
device = torch.device('cuda:3')
dataset = 'CIFAR100'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCACIFAR100_lam0.0_n100_lr0.1_lrgmm1e-05_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_CIFAR100_lr0.1_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_CIFAR100_n100_data_used50000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

Using downloaded and verified file: ../data/test_32x32.mat


/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `

0.7646
0.7649


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.761
0.7679
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &     FPR@95 &       MMC &     AUROC &     FPR@95 \\
DataSet   &           &           &            &           &           &            \\
\midrule
orig      &  0.806369 &         - &          - &  0.805886 &         - &          - \\
SVHN      &  0.437698 &  0.842599 &     0.0459 &  0.437697 &   0.84203 &     0.0459 \\
CIFAR10   &  0.522311 &   0.78657 &     0.1014 &  0.522312 &  0.785995 &     0.1016 \\
LSUN\_CR   &  0.467175 &  0.845067 &  0.0566667 &  0.467187 &  0.844972 &  0.0566667 \\
Imagenet- &  0.489895 &  0.807401 &     0.0764 &  0.483806 &  0.809774 &     0.0757 \\
Noise     &  0.532573 &  0.779823 &   0.120156 &  0.529522 &  0.781468 &   0.116328 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &     FPR@95 &       MMC &     AUROC &    FPR@95